In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import yfinance as yf

from ta.trend import sma_indicator, ema_indicator, macd_diff
from ta.momentum import rsi


# 1) ЗАГРУЗКА ЦЕН
# ============================================================
ticker = "TSLA"

df = yf.download(ticker, period="2y", interval="1h")
# === 1. Загрузка данных по акции ===
ticker = "AAPL"  # можно заменить на MSFT, TSLA и т.д.
# df = yf.download(ticker, start="2020-01-01", end="2025-10-01")


C:\Users\Данил\AppData\Local\Temp\ipykernel_21540\1911048687.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", interval="1h")


[*********************100%***********************]  1 of 1 completed


In [3]:
df.tail(50)

Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
Datetime,,,,,
2025-11-05 18:30:00+00:00,462.035400,464.769989,457.100006,457.973511,9024313
2025-11-05 19:30:00+00:00,461.630005,466.329895,460.839996,462.000000,8152952
2025-11-05 20:30:00+00:00,462.250000,462.769989,459.640106,461.620087,5650818
2025-11-06 14:30:00+00:00,451.519897,467.450012,450.929993,461.959991,20192466
2025-11-06 15:30:00+00:00,441.114990,452.260010,437.640015,451.570007,13412792
2025-11-06 16:30:00+00:00,439.998810,443.959991,435.089996,441.089996,11829453
2025-11-06 17:30:00+00:00,446.829987,447.739990,439.530701,439.970001,8344629
2025-11-06 18:30:00+00:00,451.179901,451.250000,446.299988,446.820007,9503219


In [4]:
df.reset_index(drop=True, inplace=True)
df

Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
0,139.043335,139.566666,136.860001,138.816666,52073100
1,137.253326,141.000000,136.506668,138.350006,59565300
2,136.166672,137.509995,133.886673,136.949997,59313300
3,136.029999,137.483337,134.696671,136.309998,80515800
4,147.203339,154.000000,144.336670,153.389999,183564900
...,...,...,...,...,...
1251,462.070007,466.329987,440.709991,452.049988,85573000
1252,445.910004,467.450012,435.089996,461.959991,109622900
1253,429.519989,439.359985,421.880005,437.920013,103471500


In [2]:

df = df[["Open", "High", "Low", "Close", "Volume"]].dropna()
df.reset_index(drop=True, inplace=True)

print("Загружено строк:", df.shape[0])


# ============================================================
# 2) СОЗДАНИЕ ЦЕЛЕВОГО ПРИЗНАКА (прогноз следующего Close)
# ============================================================
df["Next_Close"] = df["Close"].shift(-1)
df.dropna(inplace=True)

features = ["Open", "High", "Low", "Close", "Volume", ]
target = "Next_Close"


# ============================================================
# 3) ФУНКЦИЯ ДЛЯ ФОРМИРОВАНИЯ СКОЛЬЗЯЩЕГО ОКНА
# ============================================================
def make_sliding_window(df, features, target, window):
    X, y = [], []
    data = df[features].values
    tgt = df[target].values

    for i in range(len(df) - window):
        X.append(data[i:i + window])
        y.append(tgt[i + window])

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)




# ============================================================
# 4) TRAIN / VAL / TEST SPLIT
# ============================================================


# ============================================================
# 5) МАСШТАБИРОВАНИЕ
# ============================================================

# transform
def scale_X(X):
    X2 = X.reshape(-1, X.shape[2])
    X2 = scaler_x.transform(X2)
    return X2.reshape(X.shape)




# ============================================================
# 6) DATASET
# ============================================================
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]






class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, lstm_out):
        weights = torch.softmax(self.attn(lstm_out), dim=1)
        context = torch.sum(weights * lstm_out, dim=1)
        return context


class CNN_BiLSTM_Attention(nn.Module):
    def __init__(self, input_size, hidden_size, cnn_out=32, kernel=3):
        super().__init__()

        self.conv = nn.Conv1d(input_size, cnn_out, kernel, padding=kernel // 2)
        self.bn = nn.BatchNorm1d(cnn_out)
        self.relu = nn.ReLU()

        self.lstm = nn.LSTM(
            input_size=cnn_out,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )

        self.attention = Attention(hidden_dim=hidden_size * 2)
        self.fc = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)           # (batch, features, seq)
        x = self.relu(self.bn(self.conv(x)))
        x = x.permute(0, 2, 1)

        lstm_out, _ = self.lstm(x)
        context = self.attention(lstm_out)
        output = self.fc(context)
        return output




# ============================================================
# 8) ОБУЧЕНИЕ
# ============================================================
def train(model, loader, val_loader, epochs=30):
    for epoch in range(epochs):
        model.train()
        losses = []

        for Xb, yb in loader:
            Xb, yb = Xb.to(device), yb.to(device)

            optimizer.zero_grad()
            pred = model(Xb).flatten()

            loss = loss_fn(pred, yb)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        # validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                pred = model(Xb).flatten()
                val_losses.append(loss_fn(pred, yb).item())

        print(f"Epoch {epoch+1}/{epochs} | Train Loss={np.mean(losses):.4f} | Val Loss={np.mean(val_losses):.4f}")





# ============================================================
# 9) ОЦЕНКА
# ============================================================
def evaluate(model, X, y_true_scaled):
    model.eval()
    X = torch.tensor(X, dtype=torch.float32).to(device)

    with torch.no_grad():
        preds = model(X).cpu().numpy().flatten()

    # вернуть масштаб
    preds_real = scaler_y.inverse_transform(preds.reshape(-1, 1)).flatten()
    y_real = scaler_y.inverse_transform(y_true_scaled.reshape(-1, 1)).flatten()

    mae = mean_absolute_error(y_real, preds_real)
    rmse = mean_squared_error(y_real, preds_real) ** 0.5
    mape = np.mean(np.abs((y_real - preds_real) / y_real)) * 100
    r2 = r2_score(y_real, preds_real)

    print("\n=== METRICS ===")
    print(f"MAE:  {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²:   {r2:.4f}")

    return preds_real, y_real




Загружено строк: 1256


In [114]:
df['price_t_plus_3'] = df['Close'].shift(-3)   # если шаг = 1 час и каждый ряд = 1 час

In [3]:
df

Price,Open,High,Low,Close,Volume,Next_Close
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,
0,146.500000,150.833328,140.333328,140.419998,104499000,136.786667
1,140.029999,140.029999,132.009995,136.786667,90852600,139.043335
2,138.816666,139.566666,136.860001,139.043335,52073100,137.253326
3,138.350006,141.000000,136.506668,137.253326,59565300,136.166672
4,136.949997,137.509995,133.886673,136.166672,59313300,136.029999
...,...,...,...,...,...,...
1250,446.750000,458.000000,443.690002,456.559998,83135800,468.369995
1251,455.989990,474.070007,453.799988,468.369995,84595200,444.260010
1252,454.459991,460.220001,443.600006,444.260010,87756600,462.070007


In [3]:
WINDOW = 40
# df["Next_Close"] = df["Close"].shift(-1)
target = "Next_Close"
# def make_features(data)->pd.DataFrame:
#     data["SMA_10"] = sma_indicator(data["Close"], window=10)
#     data["EMA_10"] = ema_indicator(data["Close"], window=10)
#     data["RSI_14"] = rsi(data["Close"], window=14)
#     data["MACD"] = macd_diff(data["Close"])


#     data["DayOfWeek"] = data['Date'].dt.dayofweek
#     data["Month"] = data['Date'].dt.month

#     data["Next_Close"] = data["Close"].shift(-1)

#     data = data.dropna()

#     return data

# def normalize_yf_dataframe(df):
#     df = df.copy()
    
#     if isinstance(df.columns, pd.MultiIndex):
#         df.columns.names = ['DataType', 'Ticker']
#         df = df.stack(level=1).reset_index()
#         df = df.rename(columns={'level_1': 'Ticker'})
#     else:
#         df = df.reset_index()
#         df['Ticker'] = df.columns[1].split()[0] if ' ' in df.columns[1] else 'UNKNOWN'

#     df = df.rename(columns=str.capitalize)
#     cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
#     df = df[[c for c in cols if c in df.columns]]

#     return df

# df = normalize_yf_dataframe(df)
# df.to_csv('MSFT.csv')
# df = pd.read_csv('MSFT.csv')
# df['Date'] = pd.to_datetime(df['Date'])
# df = make_features(df)
# features = ["Open", "High", "Low", "Close",  "SMA_10", "EMA_10", "RSI_14", "MACD", "DayOfWeek", "Month"]

X, y = make_sliding_window(df, features, target, WINDOW)

print("X shape:", X.shape)  # (samples, seq_len=20, features=5)
print("y shape:", y.shape)


train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train = X[:train_size]
y_train = y[:train_size]

X_val = X[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]

X_test = X[train_size + val_size:]
y_test = y[train_size + val_size:]

scaler_x = StandardScaler()
scaler_y = StandardScaler()

# fit ONLY on train
X_train_2d = X_train.reshape(-1, X_train.shape[2])
scaler_x.fit(X_train_2d)
scaler_y.fit(y_train.reshape(-1, 1))

X_train = scale_X(X_train)
X_val   = scale_X(X_val)
X_test  = scale_X(X_test)

y_train_scaled = scaler_y.transform(y_train.reshape(-1, 1)).flatten()
y_val_scaled   = scaler_y.transform(y_val.reshape(-1, 1)).flatten()
y_test_scaled  = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

train_loader = DataLoader(TimeSeriesDataset(X_train, y_train_scaled),
                          batch_size=32, shuffle=True)
val_loader   = DataLoader(TimeSeriesDataset(X_val, y_val_scaled),
                          batch_size=32, shuffle=False)

X shape: (1215, 40, 5)
y shape: (1215,)


In [9]:
df

Price,Open,High,Low,Close,Volume,Next_Close
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA,
0,146.500000,150.833328,140.333328,140.419998,104499000,136.786667
1,140.029999,140.029999,132.009995,136.786667,90852600,139.043335
2,138.816666,139.566666,136.860001,139.043335,52073100,137.253326
3,138.350006,141.000000,136.506668,137.253326,59565300,136.166672
4,136.949997,137.509995,133.886673,136.166672,59313300,136.029999
...,...,...,...,...,...,...
1250,446.750000,458.000000,443.690002,456.559998,83135800,468.369995
1251,455.989990,474.070007,453.799988,468.369995,84595200,444.260010
1252,454.459991,460.220001,443.600006,444.260010,87756600,462.070007


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = CNN_BiLSTM_Attention(input_size=X.shape[2], hidden_size=64).to(device)

loss_fn = nn.MSELoss()


Using device: cuda


In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
train(model, train_loader, val_loader, epochs=40)


Epoch 1/40 | Train Loss=0.0479 | Val Loss=0.3995
Epoch 2/40 | Train Loss=0.0480 | Val Loss=0.5250
Epoch 3/40 | Train Loss=0.0474 | Val Loss=0.5456
Epoch 4/40 | Train Loss=0.0467 | Val Loss=0.4361
Epoch 5/40 | Train Loss=0.0417 | Val Loss=0.4674
Epoch 6/40 | Train Loss=0.0436 | Val Loss=0.5057
Epoch 7/40 | Train Loss=0.0417 | Val Loss=0.3987
Epoch 8/40 | Train Loss=0.0423 | Val Loss=0.4600
Epoch 9/40 | Train Loss=0.0430 | Val Loss=0.4581
Epoch 10/40 | Train Loss=0.0367 | Val Loss=0.4338
Epoch 11/40 | Train Loss=0.0374 | Val Loss=0.4870
Epoch 12/40 | Train Loss=0.0378 | Val Loss=0.4643
Epoch 13/40 | Train Loss=0.0335 | Val Loss=0.4766
Epoch 14/40 | Train Loss=0.0351 | Val Loss=0.5395
Epoch 15/40 | Train Loss=0.0348 | Val Loss=0.4863
Epoch 16/40 | Train Loss=0.0347 | Val Loss=0.5216
Epoch 17/40 | Train Loss=0.0334 | Val Loss=0.5489
Epoch 18/40 | Train Loss=0.0318 | Val Loss=0.5312
Epoch 19/40 | Train Loss=0.0312 | Val Loss=0.5010
Epoch 20/40 | Train Loss=0.0301 | Val Loss=0.5547
Epoch 21/

In [ ]:
y_train

In [13]:
preds, y_real = evaluate(model, X_test[:-1], y_test_scaled[:-1])




=== METRICS ===
MAE:  35.5484
RMSE: 48.1053
MAPE: 9.81%
R²:   0.4490


In [159]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

X_all = np.concatenate([X_train, X_val, X_test], axis=0)

# X_all_scaled = scale_X(X_all)

X_tensor = torch.tensor(X_all, dtype=torch.float32).to(device)
with torch.no_grad():
    y_pred_scaled = model(X_tensor).cpu().numpy().flatten()

y_pred_real = scaler_y.inverse_transform(
    y_pred_scaled.reshape(-1, 1)
).flatten()

pred_column = np.full(len(df), np.nan)


start_index = WINDOW
end_index = WINDOW + len(y_pred_real)

pred_column[start_index:end_index] = y_pred_real[:len(pred_column) - start_index]

df["Predicted_Close"] = pred_column

df


DataType,Ticker,Open,High,Low,Close,Volume,Next_Close,Predicted_Close,price_t_plus_3,Predicted_Close_t_3
0,TSLA,223.149994,224.149994,217.860001,218.259995,30510256.0,220.460297,NaN,219.990005,NaN
1,TSLA,218.289902,220.660004,218.000000,220.460297,18283175.0,218.380005,NaN,220.949997,NaN
2,TSLA,220.449997,221.237503,218.050003,218.380005,14518568.0,219.990005,NaN,221.940002,NaN
3,TSLA,218.410004,220.149994,217.639999,219.990005,11122870.0,220.949997,NaN,222.100006,NaN
4,TSLA,220.000000,221.460007,219.880005,220.949997,10069584.0,221.940002,NaN,213.980103,NaN
...,...,...,...,...,...,...,...,...,...,...
3488,TSLA,434.225006,434.859985,426.290009,428.239990,10041263.0,428.743500,438.955597,432.405914,433.686157
3489,TSLA,428.190002,429.279999,421.880005,428.743500,9926006.0,429.807587,439.668732,431.076599,431.694794
3490,TSLA,428.755005,429.890015,424.440002,429.807587,6341342.0,432.405914,431.881012,NaN,425.598785
3491,TSLA,429.809906,433.890015,429.100006,432.405914,6470793.0,431.076599,432.385529,NaN,426.487488


In [ ]:
df.to_csv('data.csv')

In [103]:
def normalize_yf_dataframe(df):
    df = df.copy()
    
    if isinstance(df.columns, pd.MultiIndex):
        df.columns.names = ['DataType', 'Ticker']
        df = df.stack(level=1).reset_index()
        df = df.rename(columns={'level_1': 'Ticker'})
    else:
        df = df.reset_index()
        df['Ticker'] = df.columns[1].split()[0] if ' ' in df.columns[1] else 'UNKNOWN'

    df = df.rename(columns=str.capitalize)
    cols = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = df[[c for c in cols if c in df.columns]]

    return df

df = normalize_yf_dataframe(df)

df.dropna(inplace=True)

df

C:\Users\Данил\AppData\Local\Temp\ipykernel_24244\4274974996.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).reset_index()


DataType,Ticker,Open,High,Low,Close,Volume
0,TSLA,223.149994,224.149994,217.860001,218.259995,30510256.0
1,TSLA,218.289902,220.660004,218.000000,220.460297,18283175.0
2,TSLA,220.449997,221.237503,218.050003,218.380005,14518568.0
3,TSLA,218.410004,220.149994,217.639999,219.990005,11122870.0
4,TSLA,220.000000,221.460007,219.880005,220.949997,10069584.0
...,...,...,...,...,...,...
3488,TSLA,434.225006,434.859985,426.290009,428.239990,10041263.0
3489,TSLA,428.190002,429.279999,421.880005,428.743500,9926006.0
3490,TSLA,428.755005,429.890015,424.440002,429.807587,6341342.0
3491,TSLA,429.809906,433.890015,429.100006,432.405914,6470793.0


In [ ]:
def add_features(df):
    df = df.copy()

    # --- Возвраты ---
    df["Return_1"] = df["Close"].pct_change()
    df["Return_2"] = df["Close"].pct_change(2)
    df["Return_6"] = df["Close"].pct_change(6)
    df["Return_12"] = df["Close"].pct_change(12)
    
    # --- Скользящие средние ---
    df["SMA_5"] = df["Close"].rolling(5).mean()
    df["SMA_10"] = df["Close"].rolling(10).mean()
    df["SMA_20"] = df["Close"].rolling(20).mean()

    # --- Volatility ---
    df["Volatility_10"] = df["Return_1"].rolling(10).std()
    df["Volatility_20"] = df["Return_1"].rolling(20).std()

    # --- Bollinger Bands ---
    df["BB_Mid"] = df["Close"].rolling(20).mean()
    df["BB_Std"] = df["Close"].rolling(20).std()
    df["BB_Upper"] = df["BB_Mid"] + 2 * df["BB_Std"]
    df["BB_Lower"] = df["BB_Mid"] - 2 * df["BB_Std"]

    # --- RSI ---
    def compute_rsi(series, period=14):
        delta = series.diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        ma_up = up.rolling(period).mean()
        ma_down = down.rolling(period).mean()
        rsi = 100 - (100 / (1 + ma_up / ma_down))
        return rsi

    df["RSI_14"] = compute_rsi(df["Close"], 14)

    # --- MACD ---
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD"] = ema12 - ema26
    df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
    df["MACD_Hist"] = df["MACD"] - df["MACD_Signal"]

    # --- Volume-based features ---
    df["Volume_Change"] = df["Volume"].pct_change()
    df["Vol_SMA_10"] = df["Volume"].rolling(10).mean()

    # Price to volume ratio
    df["Price_Volume"] = df["Close"] / (df["Volume"] + 1)

    # --- Candle features ---
    df["Candle_Body"] = df["Close"] - df["Open"]
    df["Upper_Shadow"] = df["High"] - df[["Close", "Open"]].max(axis=1)
    df["Lower_Shadow"] = df[["Close", "Open"]].min(axis=1) - df["Low"]

    # High-Low %
    df["HL_Range"] = (df["High"] - df["Low"]) / df["Low"]

    # Normalize ranges
    df["Close_Open_Ratio"] = df["Close"] / df["Open"]
    df["High_Low_Ratio"] = df["High"] / df["Low"]

    # Drop NaN rows
    df = df.dropna().reset_index(drop=True)

    return df

df = add_features(df)



DataType,Ticker,Open,High,Low,Close,Volume,Return_1,Return_2,Return_6,Return_12,...,MACD_Hist,Volume_Change,Vol_SMA_10,Price_Volume,Candle_Body,Upper_Shadow,Lower_Shadow,HL_Range,Close_Open_Ratio,High_Low_Ratio
0,UNKNOWN,188.770004,188.779999,187.955002,188.195007,3069538.0,-0.003046,-0.004444,0.004028,0.003974,...,0.194695,-0.319128,5939211.7,0.000061,-0.574997,0.009995,0.240005,0.004389,0.996954,1.004389
1,UNKNOWN,188.199997,188.559998,187.779999,188.089996,5901858.0,-0.000558,-0.003602,-0.000319,0.006228,...,0.116753,0.922719,6163049.9,0.000032,-0.110001,0.360001,0.309998,0.004154,0.999416,1.004154
2,UNKNOWN,189.929993,190.960007,189.639999,190.085403,18705132.0,0.010609,0.010045,0.003539,0.016201,...,0.183918,2.169363,7577691.5,0.000010,0.155411,0.874603,0.289993,0.006961,1.000818,1.006961
3,UNKNOWN,190.100006,190.210007,188.809998,189.009995,6893426.0,-0.005658,0.004891,-0.000582,0.008268,...,0.142120,-0.631469,7535301.4,0.000027,-1.090012,0.110001,0.199997,0.007415,0.994266,1.007415
4,UNKNOWN,189.009995,189.660004,188.720001,189.645004,4821908.0,0.003360,-0.002317,0.003227,0.011906,...,0.143049,-0.300506,7330274.1,0.000039,0.635010,0.014999,0.289993,0.004981,1.003360,1.004981


In [ ]:
df.drop(columns=['Ticker'], inplace=True)
df.

DataType,Open,High,Low,Close,Volume,Return_1,Return_2,Return_6,Return_12,SMA_5,...,MACD_Hist,Volume_Change,Vol_SMA_10,Price_Volume,Candle_Body,Upper_Shadow,Lower_Shadow,HL_Range,Close_Open_Ratio,High_Low_Ratio
0,188.770004,188.779999,187.955002,188.195007,3069538.0,-0.003046,-0.004444,0.004028,0.003974,188.907001,...,0.194695,-0.319128,5939211.7,0.000061,-0.574997,0.009995,0.240005,0.004389,0.996954,1.004389
1,188.199997,188.559998,187.779999,188.089996,5901858.0,-0.000558,-0.003602,-0.000319,0.006228,188.642001,...,0.116753,0.922719,6163049.9,0.000032,-0.110001,0.360001,0.309998,0.004154,0.999416,1.004154
2,189.929993,190.960007,189.639999,190.085403,18705132.0,0.010609,0.010045,0.003539,0.016201,188.835083,...,0.183918,2.169363,7577691.5,0.000010,0.155411,0.874603,0.289993,0.006961,1.000818,1.006961
3,190.100006,190.210007,188.809998,189.009995,6893426.0,-0.005658,0.004891,-0.000582,0.008268,188.830081,...,0.142120,-0.631469,7535301.4,0.000027,-1.090012,0.110001,0.199997,0.007415,0.994266,1.007415
4,189.009995,189.660004,188.720001,189.645004,4821908.0,0.003360,-0.002317,0.003227,0.011906,189.005081,...,0.143049,-0.300506,7330274.1,0.000039,0.635010,0.014999,0.289993,0.004981,1.003360,1.004981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3447,269.795013,272.290009,267.269989,271.579987,7855390.0,0.007120,0.001364,-0.002210,0.005964,271.158020,...,-0.011391,-0.449238,5011337.1,0.000035,1.784973,0.710022,2.525024,0.018783,1.006616,1.018783
3448,271.559998,271.970001,269.480011,269.529999,3991205.0,-0.007548,-0.000482,-0.006982,-0.002184,270.794019,...,-0.128347,-0.491915,5182183.0,0.000068,-2.029999,0.410004,0.049988,0.009240,0.992525,1.009240
3449,269.519989,269.890015,268.320007,269.109985,3276263.0,-0.001558,-0.009095,-0.008255,-0.004881,270.217993,...,-0.225925,-0.179129,5153018.9,0.000082,-0.410004,0.370026,0.789978,0.005851,0.998479,1.005851
3450,269.114990,269.140015,268.058807,268.450012,2041690.0,-0.002452,-0.004007,-0.013016,-0.004266,269.665997,...,-0.321041,-0.376824,4680969.2,0.000131,-0.664978,0.025024,0.391205,0.004033,0.997529,1.004033


In [43]:
df= df.dropna()